In [1]:
import cantera as ct
import numpy as np

In [225]:
def chemistry_initializer(oxidizer, fuel, temp_oxidizer=None, temp_fuel=None, combustion_products=None):
    
    propellant_array = [['oxidizer', oxidizer, temp_oxidizer],['fuel', fuel, temp_fuel]]
    reactants_all   = {S.name: S for S in ct.Species.list_from_file('../chem_prop/reactants.yaml')}
    gaseous_all     = {S.name: S for S in ct.Species.list_from_file('../chem_prop/gaseous_products.yaml')}
    condensed_all   = {S.name: S for S in ct.Species.list_from_file('../chem_prop/condensed_products.yaml')}
    propellant = {}
    propellant_elements = []    

    def transport_species(gaseous_species):
        transport_species = [sp for sp in gaseous_species.values() if sp.transport is not None]
        if not transport_species:
            return None
        transport = ct.Solution(thermo="ideal-gas", transport = "mixture-averaged", species = transport_species)
        transport.transport_model = "mixture-averaged"
        return transport
    
    # initialize propellants
    for type, name, temp in propellant_array:
        if name in reactants_all:
            if temp is None:
                if reactants_all[name].thermo.input_data['model'] == 'constant-cp':
                    temp = reactants_all[name].thermo.input_data['T0']
                else:
                    temp= 298.15
            propellant[type] = ct.Solution(thermo='ideal-gas', species=[reactants_all[name]])
            propellant[type].TP = temp, 2e5
            propellant[type].equilibrate('TP')
        
        elif name in gaseous_all:
            if temp is None:
                temp = 298.15
            propellant[type] = ct.Solution(thermo='ideal-gas', species=[gaseous_all[name]])
            propellant[type].TP = temp, 2e5
            propellant[type].equilibrate('TP')

        
        elif name in condensed_all:
            if temp is None:
                temp = 298.15
            propellant[type] = ct.Solution(thermo='ideal-gas', species=[condensed_all[name]])
            propellant[type].TP = temp, 2e5
            propellant[type].equilibrate('TP')

        else:
            raise ValueError(f"{name} does not exist in the thermal database.")    
        
        propellant_elements.extend(propellant[type].element_names)
    
    if combustion_products == None: 
        condensed_species = {k: v for k, v in condensed_all.items() if set(v.composition.keys()).issubset(propellant_elements)}
        gaseous_species = {k: v for k, v in gaseous_all.items() if set(v.composition.keys()).issubset(propellant_elements) and ('+' not in k and '-' not in k)}
        transport = transport_species(gaseous_species)
    else: 
        # checks if designated combustion products exist in the thermal database. if it does then filters the data
        if set(combustion_products).issubset(set(condensed_all.keys()) | set(gaseous_all.keys())):
            condensed_species = {k: v for k, v in condensed_all.items() if k in combustion_products and set(v.composition.keys()).issubset(propellant_elements)}
            gaseous_species = {k: v for k, v in gaseous_all.items() if k in combustion_products and set(v.composition.keys()).issubset(propellant_elements) and ('+' not in k and '-' not in k)}
            transport = transport_species(gaseous_species)
        else:
            invalid_species = set(combustion_products) - (set(condensed_all.keys()) | set(gaseous_all.keys()))
            raise ValueError(f"the following species {invalid_species} do not exist in the thermal database.")   
        
    # if condensed_species:
    #     condensate = ct.Solution(thermo='ideal-condensed', species=list(condensed_species.values()))
    # else: 
    #     condensate = None
    condensate = None # DELETE THIS 
    if gaseous_species:
        gas =  ct.Solution(thermo='ideal-gas', species=list(gaseous_species.values()))
    else: 
        raise ValueError("No valid gas phase combustion products of given propellants. Consider changing propllants or inputted combustion products")        

    return propellant['oxidizer'], propellant['fuel'], gas, condensate, transport

In [226]:
# test_yaml = """
# phases:
# - name: condensed
#   species:
#   - ../chem_prop/condensed_products.yaml/species: ['C(gr)', 'H2O(L)']
#   thermo: ideal-condensed
#   standard-concentration-basis: unity"""
# test_condensed = ct.Solution(yaml=test_yaml)
# test_condensed()


In [227]:
graphite_yaml = """
phases:
- name: C(gr) phase
  species:
  - ../chem_prop/condensed_products.yaml/species: ['C(gr)']
  thermo: fixed-stoichiometry
  density: 1 kg/m^3   # gives a sensible constant molar volume
  standard-concentration-basis: unity"""

water = """
phases:
- name: H2O(L) phase
  species:
  - ../chem_prop/condensed_products.yaml/species: ['H2O(L)']
  thermo: fixed-stoichiometry
  density: 5000 kg/m^3   # gives a sensible constant molar volume"""

graphite = ct.Solution(yaml=graphite_yaml)
graphite()

water = ct.Solution(yaml=water)
water()


  C(gr) phase:

       temperature   298.15 K
          pressure   1.0132e+05 Pa
           density   1 kg/m^3
  mean mol. weight   12.011 kg/kmol
   phase of matter   unspecified

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy        0.00083099         0.0099811  J
   internal energy       -1.0132e+05        -1.217e+06  J
           entropy            477.39              5734  J/K
    Gibbs function       -1.4233e+05       -1.7096e+06  J
 heat capacity c_p            710.01              8528  J/K
 heat capacity c_v            710.01              8528  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
             C(gr)                 1                 1          -0.68964


  H2O(L) phase:

       temperature   298.15 K
          pressure   1.0132e+05 Pa
           density   5000 kg/m^3
  mean mol. weigh

In [228]:
o = "O2(L)"
f = "CH4(L)"
pressure = 2e6
of = 1.5

oxidizer, fuel, gas, condensate, transport = chemistry_initializer(o, f)
molar_ratio = of / (oxidizer.mean_molecular_weight / fuel.mean_molecular_weight)
moles_ox = molar_ratio / (1 + molar_ratio)
moles_f = 1 - moles_ox

mix = ct.Mixture([(fuel, moles_f), (oxidizer, moles_ox), (gas, 0)])
mix()


************ Phase  ************
Moles:  0.5707558528428094

       temperature   111.64 K
          pressure   2e+05 Pa
           density   3.4566 kg/m^3
  mean mol. weight   16.043 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -5.5621e+06       -8.9233e+07  J
   internal energy         -5.62e+06       -9.0161e+07  J
           entropy           -352.41           -5653.7  J/K
    Gibbs function       -5.5228e+06       -8.8602e+07  J
 heat capacity c_p                 0                 0  J/K
 heat capacity c_v           -518.26           -8314.5  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
            CH4(L)                 1                 1            -95.45

************ Phase  ************
Moles:  0.4292441471571906

       temperature   111.64 K
      

In [229]:
mix.equilibrate("HP", solver="gibbs")
mix()

************ Phase  ************
Moles:  0.0

       temperature   1667.6 K
          pressure   2e+05 Pa
           density   0.23142 kg/m^3
  mean mol. weight   16.043 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -5.5621e+06       -8.9233e+07  J
   internal energy       -6.4264e+06        -1.031e+08  J
           entropy           -352.41           -5653.7  J/K
    Gibbs function       -4.9744e+06       -7.9805e+07  J
 heat capacity c_p                 0                 0  J/K
 heat capacity c_v           -518.26           -8314.5  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
            CH4(L)                 1                 1           -5.7559

************ Phase  ************
Moles:  0.0

       temperature   1667.6 K
          pressure   2e+05 Pa
     

In [236]:
transport.TPX = gas.T, gas.P, {k: v for k, v in zip(gas.species_names, gas.X) if k in transport.species_names}
        
        
# frozen transport properties
properties = {  "viscosity"             : transport.viscosity*1e4,                                                  # frozen viscosity in Pa*s
                "thermal conductivity"  : transport.thermal_conductivity*10,                                       # thermal conductivity in W/m/K
                "prandtl number"        : transport.cp * transport.viscosity / transport.thermal_conductivity   # frozen prandtl number
                }

properties

{'viscosity': 0.5370552149605982,
 'thermal conductivity': 3.0723741212803093,
 'prandtl number': 0.4806819372207381}

In [231]:
0.4806819372207381/0.5076

0.9469699314829355

In [232]:
# Defines the number of moles of each species in the mixture
if mix.n_phases == 3 or (mix.n_phases == 4 and mix.phase_moles(3) == 0):
    gas = mix.phase(2)
    condensate = None
    moles = gas.X * (1/ gas.mean_molecular_weight)
    n_elements = gas. n_elements
    num_variables = 2 * gas. n_elements + 2

elif mix.n_phases == 4: 
    gas = mix.phase(2) 
    condensate = mix.phase(3)
    moles = mix.phase_moles(2) * gas.X * (1/ gas.mean_molecular_weight) + mix.phase_moles(3) * condensate.X * (1/condensate.mean_molecular_weight)
    element_names = set(gas.element_names + condensate.element_names)
    n_elements = len(element_names)
    num_variables = 2 * n_elements + 2 * condensate.n_species + 2
else: 
    raise ValueError("Number of phases in mixture is invalid.")

# Initializing Solution Matrices table 2.3 and 2.4 in RP-1311
coeff_matrix = np.zeros((num_variables, num_variables))
right_hand_side = np.zeros(num_variables)

if not condensate: 
    # Initializes a_ij
    a_ij = np.zeros((n_elements, gas.n_species))
    for i, element in enumerate(gas.element_names): 
        for j, species in enumerate(gas.species_names):
            a_ij[i,j] = gas.n_atoms(species, element)

    # Coefficients for equation 2.56 
    for k in range(n_elements):
        for i in range(n_elements):
            coeff_matrix[k,i] = np.sum(a_ij[k,:] * a_ij[i,:] * moles)
        coeff_matrix[k, n_elements] = np.sum(a_ij[k,:] * moles)
        right_hand_side[k] = -np.sum(a_ij[k,:] * moles * gas.standard_enthalpies_RT)

    # Coefficients for equation 2.58 
    for i in range(n_elements):
        coeff_matrix[n_elements, i] = np.sum(a_ij[i, :] * moles)
    right_hand_side[n_elements] = -np.sum(moles * gas.standard_enthalpies_RT)

    # Coefficients for equation 2.64 
    for k in range(n_elements):
        for i in range(n_elements):
            coeff_matrix[n_elements+1+k,n_elements+1+i] = np.sum(a_ij[k,:] * a_ij[i,:] * moles)
        coeff_matrix[n_elements+1+k, 2*n_elements+1] = np.sum(a_ij[k,:] * moles)
        right_hand_side[n_elements+1+k] = np.sum(a_ij[k,:] * moles)

    # Coefficeints for equation 2.66
    for i in range(n_elements):
        coeff_matrix[2*n_elements+1, n_elements+1+i] = np.sum(a_ij[i, :] * moles)
    right_hand_side[2*n_elements+1] = np.sum(moles)

    # Solve for the derivatives define them based off table 2.3, 2.4 and equation 2.50 and 2.51 
    derivs = np.linalg.solve(coeff_matrix, right_hand_side)
    derivatives = { "dpi_dlnT_P"    : derivs[0 : n_elements], 
                    "dnj_dlnT_P"    : np.nan,
                    "dlnn_dlnT_P"   : derivs[n_elements], 
                    "dpi_dlnP_T"    : derivs[n_elements + 1: 2 * n_elements + 1],
                    "dnj_dlnP_T"    : np.nan,
                    "dlnn_dlnP_T"   : derivs[2 * n_elements + 1], 
                    "dlnV_dlnT_P"   : 1 + derivs[n_elements], 
                    "dlnV_dlnP_T"   : -1 + derivs[2 * n_elements + 1]}

# else: 
#     # Initializes a_ij
#     a_ij = np.zeros((n_elements, gas.n_species + condensate.n_species))
#     for i, element in enumerate(element_names): 
#         for j, species in enumerate(gas.species_names + condensate.species_names):
#             if j < gas.n_species:
#                 a_ij[i,j] = gas.n_atoms(species, element)
#             else: 
#                 a_ij[i,j] = condensate.n_atoms(species, element)
    
#     # Coefficients for equation 2.56 TODO: add terms for condensed species
#     for k in range(n_elements):
#         for i in range(n_elements):
#             coeff_matrix[k,i] = np.sum(a_ij[k, :gas.n_species] * a_ij[i,:gas.n_species] * moles)
#         coeff_matrix[k, n_elements + gas.n_species + condensate.n_species] = np.sum(a_ij[k, :gas.n_species] * moles)
#         right_hand_side[k] = -np.sum(a_ij[k,:] * moles * gas.standard_enthalpies_RT)
#     coeff_matrix[:n_elements, n_elements:gas.n_species + condensate.n_species] = a_ij[:, gas.n_species:]

#     # TODO add equation 2.57 (it is for condensed species)
#     for j in range(gas.n_species, gas.n_species + condensate.n_species):
#         right_hand_side[k] = -np.sum(a_ij[k,:] * moles * gas.standard_enthalpies_RT)
#     coeff_matrix[n_elements:n_elements+condensate.n_species, :n_elements] = a_ij.T

#     # Coefficients for equation 2.58 
#     for i in range(n_elements):
#         coeff_matrix[n_elements, i] = np.sum(a_ij[i, :] * moles)
#     right_hand_side[n_elements] = -np.sum(moles * gas.standard_enthalpies_RT)

#     # Coefficients for equation 2.64 TODO: add terms for condensed species
#     for k in range(n_elements):
#         for i in range(n_elements):
#             coeff_matrix[n_elements+1+k, n_elements+1+i] = np.sum(a_ij[k,:] * a_ij[i,:] * moles)
#         for j in range(gas.n_species, gas.n_species + condensate.n_species):
#             coeff_matrix[k, n_elements+j] = a_ij[k, j]
#         coeff_matrix[n_elements+1+k, 2*n_elements+]
#         right_hand_side[n_elements+1+k] = np.sum(a_ij[k,:] * moles)

In [233]:
def __get_thermo_derivatives(mix):
    '''
    This is an internal method not meant for use outside the class.
    given a mixture of gases & condensed species this method will find thermodynamic derivatives.
    These derivatives can then be used to calculate thermodynamic properties, or create interpolation/extrapolation of engine properties 

    The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.
    The equations used are 2.50, 2.51, 2.56, 2.57, 2.58, 2.64, 2.65, 2.66 

    TODO: add condensed species to implementation
    TODO: explore interpolation of engine properties using derivatives
    
    Parameters
    --------------------------------------------------
    mixture : Cantera Mixture
        A solution or mixture of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
    
    Returns
    --------------------------------------------------
    Derivatives : dictionary:
        A dictionary consisting of the derivatives this function calculates. The keys and descritions of said dictionaries are as follows
    
        dpi_dlnT_P : list
            A list of the derivative of pi with respect to ln(T) at constant pressure for each element, where pi is -lambda/RT, 
            and lambda is the langrangian multiplier
        dlnn_dlnT_P : float
            The derivative of ln(n) with respect to ln(T) at constant pressure. Where n is the number of moles of gas
        dpi_dlnP_T : list
            A list of the derivative of pi with respect to ln(P) at constant temperature for each element.
        dlnn_dlnP_T : float
            The derivative of ln(n) with respect to ln(P) at constant tempearture.
        dlnV_dlnT_P : float
            The derivative of ln(V) with respect to ln(T) at constant pressure.
        dlnV_dlnP_T : float
            The derivative of ln(V) with respect to ln(P) at constant tempearture.
    '''
                
    # Defines the number of moles of each species in the mixture
    if mix.n_phases == 3 or (mix.n_phases == 4 and mix.phase_moles(3) == 0):
        gas = mix.phase(2)
        condensate = None
        moles = gas.X * (1/ gas.mean_molecular_weight)
        n_elements = gas. n_elements
        num_variables = 2 * gas. n_elements + 2

    elif mix.n_phases == 4: 
        gas = mix.phase(2) 
        condensate = mix.phase(3)
        
        moles = mix.phase_moles(2) * gas.X * (1/ gas.mean_molecular_weight) + mix.phase_moles(3) * condensate.X * (1/condensate.mean_molecular_weight)
        element_names = set(gas.element_names + condensate.element_names)
        n_elements = len(element_names)
        num_variables = 2 * n_elements + 2 * condensate.n_species + 2
    else: 
        raise ValueError("Number of phases in mixture is invalid.")

    # Initializing Solution Matrices table 2.3 and 2.4 in RP-1311
    coeff_matrix = np.zeros((num_variables, num_variables))
    right_hand_side = np.zeros(num_variables)

    if not condensate: 
        # Initializes a_ij
        a_ij = np.zeros((n_elements, gas.n_species))
        for i, element in enumerate(gas.element_names): 
            for j, species in enumerate(gas.species_names):
                a_ij[i,j] = gas.n_atoms(species, element)

        # Coefficients for equation 2.56 
        for k in range(n_elements):
            for i in range(n_elements):
                coeff_matrix[k,i] = np.sum(a_ij[k,:] * a_ij[i,:] * moles)
            coeff_matrix[k, n_elements] = np.sum(a_ij[k,:] * moles)
            right_hand_side[k] = -np.sum(a_ij[k,:] * moles * gas.standard_enthalpies_RT)

        # Coefficients for equation 2.58 
        for i in range(n_elements):
            coeff_matrix[n_elements, i] = np.sum(a_ij[i, :] * moles)
        right_hand_side[n_elements] = -np.sum(moles * gas.standard_enthalpies_RT)

        # Coefficients for equation 2.64 
        for k in range(n_elements):
            for i in range(n_elements):
                coeff_matrix[n_elements+1+k,n_elements+1+i] = np.sum(a_ij[k,:] * a_ij[i,:] * moles)
            coeff_matrix[n_elements+1+k, 2*n_elements+1] = np.sum(a_ij[k,:] * moles)
            right_hand_side[n_elements+1+k] = np.sum(a_ij[k,:] * moles)

        # Coefficeints for equation 2.66
        for i in range(n_elements):
            coeff_matrix[2*n_elements+1, n_elements+1+i] = np.sum(a_ij[i, :] * moles)
        right_hand_side[2*n_elements+1] = np.sum(moles)

        # Solve for the derivatives define them based off table 2.3, 2.4 and equation 2.50 and 2.51 
        derivs = np.linalg.solve(coeff_matrix, right_hand_side)
        derivatives = { "dpi_dlnT_P"    : derivs[0 : n_elements], 
                        "dnj_dlnT_P"    : np.nan,
                        "dlnn_dlnT_P"   : derivs[n_elements], 
                        "dpi_dlnP_T"    : derivs[n_elements + 1: 2 * n_elements + 1],
                        "dnj_dlnP_T"    : np.nan,
                        "dlnn_dlnP_T"   : derivs[2 * n_elements + 1], 
                        "dlnV_dlnT_P"   : 1 + derivs[n_elements], 
                        "dlnV_dlnP_T"   : -1 + derivs[2 * n_elements + 1]}

    else: 
        # Initializes a_ij
        a_ij = np.zeros((n_elements, gas.n_species + condensate.n_species))
        for i, element in enumerate(element_names): 
            for j, species in enumerate(gas.species_names + condensate.species_names):
                if j < gas.n_species:
                    a_ij[i,j] = gas.n_atoms(species, element)
                else: 
                    a_ij[i,j] = condensate.n_atoms(species, element)
        
        # Coefficients for equation 2.56 TODO: add terms for condensed species
        for k in range(n_elements):
            for i in range(n_elements):
                coeff_matrix[k,i] = np.sum(a_ij[k, :gas.n_species] * a_ij[i,:gas.n_species] * moles)
            coeff_matrix[k, n_elements + gas.n_species + condensate.n_species] = np.sum(a_ij[k, :gas.n_species] * moles)
            right_hand_side[k] = -np.sum(a_ij[k,:] * moles * gas.standard_enthalpies_RT)
        coeff_matrix[:n_elements, n_elements:gas.n_species + condensate.n_species] = a_ij[:, gas.n_species:]

        # TODO add equation 2.57 (it is for condensed species)
        for j in range(gas.n_species, gas.n_species + condensate.n_species):
            right_hand_side[k] = -np.sum(a_ij[k,:] * moles * gas.standard_enthalpies_RT)
        coeff_matrix[n_elements:n_elements+condensate.n_species, :n_elements] = a_ij.T

        # Coefficients for equation 2.58 
        for i in range(n_elements):
            coeff_matrix[n_elements, i] = np.sum(a_ij[i, :] * moles)
        right_hand_side[n_elements] = -np.sum(moles * gas.standard_enthalpies_RT)

        # Coefficients for equation 2.64 TODO: add terms for condensed species
        for k in range(n_elements):
            for i in range(n_elements):
                coeff_matrix[n_elements+1+k, n_elements+1+i] = np.sum(a_ij[k,:] * a_ij[i,:] * moles)
            for j in range(gas.n_species, gas.n_species + condensate.n_species):
                coeff_matrix[k, n_elements+j] = a_ij[k, j]
            coeff_matrix[n_elements+1+k, 2*n_elements+]
            right_hand_side[n_elements+1+k] = np.sum(a_ij[k,:] * moles)

        

        # TODO add equation 2.65 (it is for condensed species)






    return derivatives

SyntaxError: invalid syntax (940507683.py, line 137)

In [ ]:
def __get_thermo_properties(gas, dpi_dlnT_P, dlnn_dlnT_P, dlnV_dlnT_P, dlnV_dlnP_T):
        '''
        This is an internal method not meant for use outside the class.
        given a mixture of gases & condensed species, as well as certain thermdynamic derivatives of said mixture, this function will find 
        the thermodynamic properties of the mixture

        The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.

        TODO: add condensed species to implementation
        TODO: add capability to solve for thermal conductivity, viscocity, and prandtl number  

        Parameters
        --------------------------------------------------
        mixture : Cantera Mixture
            A solution or mixture of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
        dpi_dlnT_P : list
            A list of the derivative of pi with respect to ln(T) at constant pressure for each element, where pi is -lambda/RT, 
            and lambda is the langrangian multiplier
        dlnn_dlnT_P : float
            The derivative of ln(n) with respect to ln(T) at constant pressure. Where n is the number of moles of gas.
        dlnV_dlnT_P : float
            The derivative of ln(V) with respect to ln(T) at constant pressure.
        dlnV_dlnP_T : float
            The derivative of ln(V) with respect to ln(P) at constant tempearture.
            
        Returns
        --------------------------------------------------
        properties : dictionaries
            A dictionary consisting of the properties this function calculates. The keys and descritions of said dictionaries are as follows

            Pressure : float
            Temperature : float
            density : float
            specific_volume : float
            enthalpy : float
            internal_energy : float
            gibbs : float
            entropy : float
            molar_mass : float
            c_p : float
                the specific heat at constant pressure
            c_v : float
                the specific heat at constant volume
            gamma : float
                the specific heat ratio
            gamma_s : float
                defined as derivative ln(P) with respect to ln(rho) at constant entropy per equation 2.71 in RP-1311
            speed_sound : float
                the speed of sound in the mixture
        '''

        # Defines the number of moles of each species in the micture
        moles = gas.X * (1/ gas.mean_molecular_weight)

        # Initializes a_ij
        a_ij = np.zeros((gas.n_elements, gas.n_species))
        for i, element in enumerate(gas.element_names): 
            for j, species in enumerate(gas.species_names):
                a_ij[i,j] = gas.n_atoms(species, element)

        # Finds specific heat at constant pressure based on equation 2.59 TODO: add terms for condensed species
        c_p =  ct.gas_constant * (
            np.sum([dpi_dlnT_P[i] * np.sum(a_ij[i,:] * moles * gas.standard_enthalpies_RT) for i in range(gas.n_elements)]) +
            np.sum(moles * gas.standard_enthalpies_RT) * dlnn_dlnT_P +
            np.sum(moles * gas.standard_cp_R) +
            np.sum(moles * gas.standard_enthalpies_RT**2)
        )

        # Finds specifc heat at constant volume, based on equation 2.70, specific heat ratio, The isentropic exponent based on equation 2.73, 
        # and speed of sound based on equation 2.74
        c_v = c_p + gas.P * gas.v / gas.T * dlnV_dlnT_P**2 / dlnV_dlnP_T
        gamma = c_p / c_v
        gamma_s = -gamma/dlnV_dlnP_T
        speed_sound = np.sqrt(ct.gas_constant * gas.T * gamma_s/gas.mean_molecular_weight)

        properties = {  
                        "pressure"          : gas.P,
                        "temperature"       : gas.T,
                        "density"           : gas.density_mass,
                        "specific volume"   : gas.volume_mass,
                        "enthalpy"          : gas.enthalpy_mass,
                        "internal energy"   : gas.int_energy_mass,
                        "gibbs"             : gas.gibbs_mass,
                        "entropy"           : gas.entropy_mass,
                        "molar mass"        : gas.mean_molecular_weight,
                        "c_p"               : c_p,
                        "c_v"               : c_v,
                        "gamma"             : gamma,                                                                            
                        "gamma_s"           : gamma_s,    
                        "speed sound"       : speed_sound          
                        }

        return properties

In [ ]:
full_species = {S.name: S for S in ct.Species.list_from_file('nasa_gas.yaml')}
reactants_all   = {S.name: S for S in ct.Species.list_from_file('../chem_prop/reactants.yaml')}

temp_ox = 90.17
temp_fuel = 111.64
o = "O2(L)"
f = "CH4(L)"
pressure = 2e6
of = 1.5

fuel = ct.Solution(thermo='ideal-gas', species=[reactants_all[f]])
ox = ct.Solution(thermo='ideal-gas', species=[reactants_all[o]])
fuel.TP = temp_fuel, pressure
ox.TP = temp_ox, pressure
molar_ratio = of / (ox.mean_molecular_weight / fuel.mean_molecular_weight)
moles_ox = molar_ratio / (1 + molar_ratio)
moles_f = 1 - moles_ox

species_list = ['C2H2,acetylene', 'O2', 'H2O', 'CO', 'CO2', 'H2O2', 'OH', 'H', 'CH4', 'C2H4', 'H2']
species = {S.name: S for S in ct.Species.list_from_file('../chem_prop/gaseous_products.yaml')}


gas_chamber = ct.Solution(thermo='ideal-gas', species=[species[S] for S in species_list] )

mix = ct.Mixture([(fuel, moles_f), (ox, moles_ox), (gas_chamber, 0)])
mix.equilibrate('HP')

gas_chamber()


In [ ]:
__get_thermo_derivatives(mix)